In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Load the dataset

In [ ]:
# Load Data
from google.colab import files
uploaded = files.upload()

# Assuming 'mallika_combined.pkl' is the name of the uploaded file
data = pd.read_pickle('final_combined_dataset.pickle')




Saving final_combined_dataset.pickle to final_combined_dataset.pickle


In [ ]:
# Preprocess Data: Convert boards into 6x7x2 representation
def preprocess_data(data):
    boards = []
    labels = []
    for _, item in data.iterrows():
        board = np.array(item['board'])

        # Convert to 6x7x2 format
        if board.shape == (6, 7, 2):
            board_6x7x2 = board
        else:
            board_6x7x2 = np.zeros((6, 7, 2))
            board_6x7x2[:, :, 0] = (board == 1).astype(int)
            board_6x7x2[:, :, 1] = (board == -1).astype(int)

        label = item['recommended_column']
        boards.append(board_6x7x2)
        labels.append(label)

    boards = np.array(boards)
    labels = np.array(labels)

    # One-hot encode labels (7 possible moves)
    labels = tf.keras.utils.to_categorical(labels, num_classes=7)
    return boards, labels

# Get processed data
boards, labels = preprocess_data(data)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(boards, labels, test_size=0.2, random_state=42)

# Reshape Data for Transformer (Flatten the board)
num_samples, n, m, channels = X_train.shape  # (num_samples, 6, 7, 2)
X_train = X_train.reshape(num_samples, n * m, channels)
X_val = X_val.reshape(X_val.shape[0], n * m, channels)

# Debugging Step: Check shape before proceeding
print(f"X_train shape: {X_train.shape}")  # Expected: (num_samples, 42, 2)
print(f"y_train shape: {y_train.shape}")  # Expected: (num_samples, 7)

X_train shape: (1003039, 42, 2)
y_train shape: (1003039, 7)


In [ ]:
import tensorflow as tf

class PositionalIndex(tf.keras.layers.Layer):
    def call(self, x):
        bs = tf.shape(x)[0]  # Extract batch size
        number_of_vectors = tf.shape(x)[1]  # Count the number of vectors (should be m*n)
        indices = tf.range(number_of_vectors)  # Index for each vector
        indices = tf.expand_dims(indices, 0)  # Reshape appropriately
        return tf.tile(indices, [bs, 1])  # Repeat for each batch


class ClassTokenIndex(tf.keras.layers.Layer):
    def call(self, x):
        bs = tf.shape(x)[0]  # Extract batch size
        number_of_vectors = 1  # We want just 1 vector for the class token
        indices = tf.range(number_of_vectors)  # Index for the vector
        indices = tf.expand_dims(indices, 0)  # Reshape appropriately
        return tf.tile(indices, [bs, 1])  # Repeat for each batch


## Transformer

In [ ]:
def build_ViT(n,m,block_size,hidden_dim,num_layers,num_heads,key_dim,value_dim,mlp_dim,dropout_rate,num_classes):
    # n is number of rows of blocks
    # m is number of cols of blocks
    # block_size is number of pixels (with rgb) in each block
    inp = tf.keras.layers.Input(shape=(n*m,block_size))
    mid = tf.keras.layers.Dense(hidden_dim)(inp) # transform to vectors with different dimension
    # the positional embeddings
    inp2 = PositionalIndex()(inp)
    emb = tf.keras.layers.Embedding(input_dim=n*m, output_dim=hidden_dim)(inp2) # learned positional embedding for each of the n*m possible possitions
    mid = tf.keras.layers.Add()([mid, emb]) # for some reason, tf.keras.layers.Add causes an error, but + doesn't?
    # create and append class token to beginning of all input vectors
    tokenInd = ClassTokenIndex()(mid)
    token = tf.keras.layers.Embedding(input_dim=1, output_dim=hidden_dim)(tokenInd)
    mid = tf.keras.layers.Concatenate(axis=1)([token, mid])

    for l in range(num_layers): # how many Transformer Head layers are there?
        ln  = tf.keras.layers.LayerNormalization()(mid) # normalize
        mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,value_dim=value_dim)(ln,ln,ln) # self attention!
        add = tf.keras.layers.Add()([mid,mha]) # add and norm
        ln  = tf.keras.layers.LayerNormalization()(add)
        den = tf.keras.layers.Dense(mlp_dim,activation='gelu')(ln) # maybe should be relu...who knows...
        den = tf.keras.layers.Dropout(dropout_rate)(den) # regularization
        den = tf.keras.layers.Dense(hidden_dim)(den) # back to the right dimensional space
        den = tf.keras.layers.Dropout(dropout_rate)(den)
        mid = tf.keras.layers.Add()([den,add]) # add and norm again

    fl = mid[:,0,:] # just grab the class token for each image in batch
    ln = tf.keras.layers.LayerNormalization()(fl)
    clas = tf.keras.layers.Dense(num_classes,activation='softmax')(ln) # probability that the image is in each category
    mod = tf.keras.models.Model(inp,clas)
    mod.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return mod

In [ ]:
# Model hyperparameters
n = 6
m = 7
block_size = 2
hidden_dim = 32
num_layers = 4
num_heads = 4
key_dim = hidden_dim // num_heads  # Good practice for key_dim to be hidden_dim//num_heads
value_dim = key_dim * 2
mlp_dim = hidden_dim
dropout_rate = 0.1
num_classes = 7  # Output classes for classification

# Build the Transformer model
trans = build_ViT(n, m, block_size, hidden_dim, num_layers, num_heads,
                  key_dim, value_dim, mlp_dim, dropout_rate, num_classes)

# Display model summary
trans.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 42, 2)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_index          │ (None, 42)             │              0 │ input_layer[0][0]      │
│ (PositionalIndex)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 42, 32)         │             96 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 42, 32)         │          1,344 │ positional_index[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 42, 32)         │              0 │ dense[0][0],           │
│                           │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ class_token_index         │ (None, 1)              │              0 │ add[0][0]              │
│ (ClassTokenIndex)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 32)          │             32 │ class_token_index[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 43, 32)         │              0 │ embedding_1[0][0],     │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 43, 32)         │             64 │ concatenate[0][0]      │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 43, 32)         │          6,304 │ layer_normalization[0… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization[0… │
│                           │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 43, 32)         │              0 │ concatenate[0][0],     │
│                           │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 43, 32)         │             64 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 43, 32)         │          1,056 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 43, 32)         │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 43, 32)         │          1,056 │ dropout_1[0][0]        │
├──────────────────────

 Total params: 35,943 (140.40 KB)

 Trainable params: 35,943 (140.40 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(f"X_train shape: {X_train.shape}")  # Expected: (num_samples, 42, 2)
print(f"y_train shape: {y_train.shape}")  # Expected: (num_samples, 7)


NameError: name 'X_train' is not defined

In [ ]:
trans.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Stop if validation loss doesn't improve
    patience=5,          # Number of epochs to wait before stopping
    restore_best_weights=True  # Restore model to the best state
)

history = trans.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,  # Adjust based on performance
                    batch_size=64)  # Tune based on memory and dataset size


Epoch 1/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 172s 9ms/step - accuracy: 0.3341 - loss: 1.5993 - val_accuracy: 0.4088 - val_loss: 1.4384
Epoch 2/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 179s 9ms/step - accuracy: 0.4181 - loss: 1.4246 - val_accuracy: 0.4388 - val_loss: 1.3820
Epoch 3/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 141s 9ms/step - accuracy: 0.4494 - loss: 1.3621 - val_accuracy: 0.4679 - val_loss: 1.3241
Epoch 4/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 147s 9ms/step - accuracy: 0.4683 - loss: 1.3227 - val_accuracy: 0.4734 - val_loss: 1.3091
Epoch 5/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 146s 9ms/step - accuracy: 0.4805 - loss: 1.2948 - val_accuracy: 0.4930 - val_loss: 1.2649
Epoch 6/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 201s 9ms/step - accuracy: 0.4894 - loss: 1.2715 - val_accuracy: 0.4988 - val_loss: 1.2491
Epoch 7/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 199s 9ms/step - accuracy: 0.4975 - loss: 1.2515 - val_accuracy: 0.5037 - val_loss: 1.2406
Epoch 8/100
15673/15673 ━━━━━━━━━━━━━━━━━━━━ 130s 8ms/s

## Test

In [ ]:
# plot results
trans.evaluate(X_val, y_val)

In [ ]:
# export model results
trans.save('transformer_model.h5')